In [138]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy
# import tflearn
# import tensorflow as tf
from tensorflow import keras
import random
import json
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [139]:
#open the json and store it
with open("intents.json") as intents:
    intent_data = json.load(intents)
# intent_data

In [140]:
#will hold all words
all_words = []

#will hold all possible intent tags
all_labels = []

#will hold all pattern data, each pattern in a list of itself
all_patterns = []

#will hold the type of intent of the corresponding word in all_patterns
all_responses = []

In [141]:
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer() 

In [142]:
#loop through the intent dictionary
for intent in intent_data['intents']:
    #loop through each pattern in each patterns list
    for pattern in intent['patterns']:
        #tokenize basically separates each sentence into individual words
        words = nltk.word_tokenize(pattern)
        #add all the words into all words
        all_words.extend(words)
        all_patterns.append(words)
        all_responses.append(intent['tag'])
        
    if intent['tag'] not in all_labels:
        all_labels.append(intent['tag'])

In [143]:
print(all_words[:5])
print(all_labels)
print(all_patterns[:5])
print(all_responses[:5])

['Hi', 'How', 'are', 'you', 'hey']
['greeting', 'goodbye', 'name', 'name_yes', 'name_no', 'thanks', 'first', 'not_certain_condition', 'lc_initial', 'lc_main', 'bc_initial', 'bc_main', 'sc_main']
[['Hi'], ['How', 'are', 'you'], ['hey'], ['yo'], ['Is', 'anyone', 'there']]
['greeting', 'greeting', 'greeting', 'greeting', 'greeting']


In [144]:
#removes the ends of the words. Basically reducing the words to their root type
#for example if there was a word that is "Whats" it removes the "s" returning only "what"
#the reason for doing this is to get the plain meaning of the word whilst ignoring any unnecessary
#additions that might confuse the model, making it to be able to generalize more
all_words = [lemmatizer.lemmatize(word.lower()) for word in all_words]
all_words = [stemmer.stem(word.lower()) for word in all_words if word]
print(all_words)
print(len(all_words))

['hi', 'how', 'are', 'you', 'hey', 'yo', 'is', 'anyon', 'there', 'hello', 'good', 'day', 'what', 'up', 'sup', 'cya', 'see', 'you', 'later', 'goodby', 'im', 'leav', 'have', 'a', 'good', 'day', 'bye', 'what', 'is', 'your', 'name', 'what', 'should', 'i', 'call', 'you', 'what', 'your', 'name', 'name', 'your', 'name', 'what', 'are', 'you', 'who', 'are', 'you', 'yes', 'haha', 'yes', 'yep', 'nice', 'inde', 'yup', 'yea', 'nah', 'no', 'nope', 'not', 'realli', 'ew', 'cring', 'i', 'do', 'not', 'it', 'doe', 'not', 'thank', 'thank', 'you', 'that', 'is', 'help', 'awesom', 'thank', 'thank', 'for', 'help', 'me', 'sick', 'i', 'do', 'not', 'feel', 'good', 'not', 'well', 'i', 'am', 'not', 'feel', 'well', 'i', 'feel', 'sick', 'someth', 'is', 'wrong', 'with', 'me', 'i', 'do', 'not', 'think', 'i', 'am', 'of', 'perfect', 'health', 'sore', 'bump', 'lump', 'i', 'have', 'a', 'sore', 'skin', 'itch', 'my', 'skin', 'itch', 'i', 'have', 'a', 'bump', 'i', 'have', 'a', 'lump', 'im', 'bleed', 'blood', 'i', 'have', 'bl

In [145]:
#remove any duplicate words, and sort for easiness
all_words = sorted(list(set(all_words)))
len(all_words)

132

In [146]:
all_labels = sorted(all_labels)
print(len(all_labels))

13


In [147]:
#A neural network cannot interpret these Strings
#However, they can be one-hot encoded to numbers
#One hot encoding - Bag of Words (if word is there - "hot", represented with a 1)
#the mapping is [the, she, he, him, they, was, a, guy, person]
#one-hot representation -> [0, 0, 1, 0, 0, 1, 1, 0, 1], for the sentence "he was a person"
#We'll use this representation for each sentence, using {all_words} as its list to mapped against

#training and output list
training = []
output = []

#creating a list of all 0's  to use as a starting point
out_empty = [0 for _ in range(len(all_labels))]
# out_empty -> [0,0,0,0,0,0]

for index, pattern in enumerate(all_patterns):
    #the bag of words
    bag = []
    words = [lemmatizer.lemmatize(word.lower()) for word in pattern]
    words = [stemmer.stem(word.lower()) for word in words]
    
    for word in all_words:
        #loop and check whether each word consists
        if word in words:
            #if it does append 1
            bag.append(1)
        else:
            bag.append(0)
    
    #create a copy of out_empty
    output_row = out_empty[:]
    
    #set the position of the tag equal to 1
    output_row[all_labels.index(all_responses[index])] = 1
    training.append(bag)
    output.append(output_row)
    
#convert to arrays, for tflearn to accept
training = numpy.array(training)
output = numpy.array(output)
#the data is now ready to be used to train

In [148]:
# #input data size - bag of words length - tflearn model
# net = tflearn.input_data(shape=[None, len(training[0])])
# #hidden layers
# net = tflearn.fully_connected(net, 8)
# net = tflearn.fully_connected(net, 8)
# #output layer, size equal to number of possibilities
# net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
# net = tflearn.regression(net)
# #Regular deep neural network
# model = tflearn.DNN(net)

# print(training.shape)
# print(output.shape)

In [149]:
# tensorflow keras model
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=[len(training[0])]))
for _ in range(5):
    model.add(keras.layers.Dense(128))
model.add(keras.layers.Dropout(0.4))
for _ in range(5):
    model.add(keras.layers.Dense(128))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(len(output[0]), activation='softmax'))

In [150]:
# try:
#     model = keras.models.load_model('chatbot.h5')
# except:
model.compile(optimizer=keras.optimizers.Adam(lr=1e-4), metrics=['accuracy'], loss='categorical_crossentropy')
model.fit(training, output, epochs=1000, batch_size=128)

# model = keras.models.load_model('chatbot.h5')

Epoch 1/1000
1/1 [==============================] - 1s 554ms/step - loss: 2.5674 - accuracy: 0.1207
Epoch 2/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.5735 - accuracy: 0.0948
Epoch 3/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.5170 - accuracy: 0.1034
Epoch 4/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.5362 - accuracy: 0.1207
Epoch 5/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.5066 - accuracy: 0.1552
Epoch 6/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.4891 - accuracy: 0.0776
Epoch 7/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.4592 - accuracy: 0.1121
Epoch 8/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.4214 - accuracy: 0.1897
Epoch 9/1000
1/1 [==============================] - 0s 7ms/step - loss: 2.4077 - accuracy: 0.2414
Epoch 10/1000
1/1 [==============================] - 0s 6ms/step - loss: 2.3581 - accuracy: 0.2241
Epoch 11/1000
1/1

1/1 [==============================] - 0s 7ms/step - loss: 0.8840 - accuracy: 0.7241
Epoch 84/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.8505 - accuracy: 0.7500
Epoch 85/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.8599 - accuracy: 0.7672
Epoch 86/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7990 - accuracy: 0.8103
Epoch 87/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.8247 - accuracy: 0.7328
Epoch 88/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7570 - accuracy: 0.8276
Epoch 89/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7548 - accuracy: 0.7759
Epoch 90/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7392 - accuracy: 0.8362
Epoch 91/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7371 - accuracy: 0.8276
Epoch 92/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7394 - accuracy: 0.8017
Epoch 93/1000
1/1 [=====

1/1 [==============================] - 0s 8ms/step - loss: 0.1009 - accuracy: 0.9914
Epoch 165/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.1129 - accuracy: 0.9828
Epoch 166/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.1237 - accuracy: 0.9914
Epoch 167/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.1331 - accuracy: 0.9655
Epoch 168/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.1289 - accuracy: 0.9741
Epoch 169/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.1139 - accuracy: 0.9741
Epoch 170/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.1199 - accuracy: 0.9741
Epoch 171/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.1016 - accuracy: 0.9828
Epoch 172/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.0999 - accuracy: 0.9741
Epoch 173/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.1214 - accuracy: 0.9828
Epoch 174/1000


1/1 [==============================] - 0s 6ms/step - loss: 0.0315 - accuracy: 1.0000
Epoch 247/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0290 - accuracy: 1.0000
Epoch 248/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0269 - accuracy: 1.0000
Epoch 249/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0471 - accuracy: 0.9914
Epoch 250/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0207 - accuracy: 1.0000
Epoch 251/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0193 - accuracy: 1.0000
Epoch 252/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0301 - accuracy: 1.0000
Epoch 253/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0388 - accuracy: 1.0000
Epoch 254/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0375 - accuracy: 0.9828
Epoch 255/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0219 - accuracy: 1.0000
Epoch 256/1000


1/1 [==============================] - 0s 7ms/step - loss: 0.0202 - accuracy: 1.0000
Epoch 329/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0095 - accuracy: 1.0000
Epoch 330/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 331/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0226 - accuracy: 0.9914
Epoch 332/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0139 - accuracy: 1.0000
Epoch 333/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 334/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0137 - accuracy: 1.0000
Epoch 335/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0138 - accuracy: 1.0000
Epoch 336/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 337/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 338/1000


...

Epoch 656/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 657/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 658/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 659/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 660/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 661/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 662/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 663/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 664/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 665/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0017 - accuracy: 1.0000


1/1 [==============================] - 0s 7ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 738/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 739/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 740/1000
1/1 [==============================] - 0s 6ms/step - loss: 4.9875e-04 - accuracy: 1.0000
Epoch 741/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 742/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 743/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 744/1000
1/1 [==============================] - 0s 6ms/step - loss: 6.5764e-04 - accuracy: 1.0000
Epoch 745/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 746/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 7

1/1 [==============================] - 0s 7ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 819/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.5271e-04 - accuracy: 1.0000
Epoch 820/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 821/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 822/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 823/1000
1/1 [==============================] - 0s 6ms/step - loss: 9.1147e-04 - accuracy: 1.0000
Epoch 824/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 825/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 826/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 827/1000
1/1 [==============================] - 0s 6ms/step - loss: 4.9628e-04 - accuracy: 1.0000
Epo

Epoch 899/1000
1/1 [==============================] - 0s 6ms/step - loss: 9.7567e-04 - accuracy: 1.0000
Epoch 900/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 901/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 902/1000
1/1 [==============================] - 0s 6ms/step - loss: 7.3268e-04 - accuracy: 1.0000
Epoch 903/1000
1/1 [==============================] - 0s 6ms/step - loss: 9.0212e-04 - accuracy: 1.0000
Epoch 904/1000
1/1 [==============================] - 0s 7ms/step - loss: 6.9596e-04 - accuracy: 1.0000
Epoch 905/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 906/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0092 - accuracy: 0.9914
Epoch 907/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 908/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0011 - a

1/1 [==============================] - 0s 7ms/step - loss: 8.6317e-04 - accuracy: 1.0000
Epoch 980/1000
1/1 [==============================] - 0s 7ms/step - loss: 7.8240e-04 - accuracy: 1.0000
Epoch 981/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 982/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.8130e-04 - accuracy: 1.0000
Epoch 983/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 984/1000
1/1 [==============================] - 0s 7ms/step - loss: 3.9604e-04 - accuracy: 1.0000
Epoch 985/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 986/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 987/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 988/1000
1/1 [==============================] - 0s 7ms/step - loss: 3.7288e-04 - accuracy: 1.

In [155]:
model.save("chatbot.h5")

In [152]:
#convert the input text into bag of words
def bag_of_words(text, all_words):
    bag = [0 for _ in range(len(all_words))]
                            
    text_words = nltk.word_tokenize(text)
    text_words = [lemmatizer.lemmatize(word.lower()) for word in text_words]
    text_words = [stemmer.stem(word.lower()) for word in text_words]
    
    for wrd in text_words:
        for index, word in enumerate(all_words):
            if word == wrd:
                bag[index] = 1
    
    return numpy.array(bag)

In [153]:
def chat(username):
    print(f'Hi {username}, how can I help you today?')
    
    #Reset context on start, due to there being no context
    context = None
    #default responses if no proper valid match
    default_responses = [
    "Sorry, can't understand you, I am not perfect :'(", "Please give me more info :(", "Not sure I understand :("]
    
    while True:
        user_input = str(input("You: ")).lower()
        if user_input == 'quit':
            break
    
        #will hold a list of probabilities (softmax)
        print(user_input)
        bag = bag_of_words(user_input, all_words)
        print(bag.shape)
        bag = bag.reshape(1, -1)
        print(bag.shape)
        results = model.predict([bag])[0]
        print(results)
        #will return index of highest probability
        result_index = numpy.argmax(results)
        print(result_index)
        #Corresponding tag of prediction
        result_tag = all_labels[result_index]
           
#         print("Prediction Probability", results[result_index])
            
        #only if the model is quite confident do this
        if results[result_index] > 0.8:
            #break loop if end
            if result_tag == 'goodbye' or result_tag == 'thanks':
                responses = intent_data['intents'][1]['responses'] if result_tag == 'goodbye' else intent_data['intents'][5]['responses']
                print("CHANCO: " + random.choice(responses))
                break
            
            for intent in intent_data['intents']:
                #predicted intent
                if intent['tag'] == result_tag:
                    print(result_tag)
#                     print('context_filter' in intent and intent['context_filter'] == context)

                    #this if condition checks to see whether the context_filter intent has the same value as the context of the
                    #intent it is referring to, if so it means that the context_filter intent will be given authority to provide a response
                    if 'context_filter' not in intent or 'context_filter' in intent and intent['context_filter'] == context:
                        #responses of corresponding intent
                        responses = intent['responses']
                        
                        #does the current intent have context
                        if 'context' in intent:
                            context = intent['context']
                        else:
                            context = None

                        #choose some random response
                        print("CHANCO: " + random.choice(responses))

                    #if user enters smth directly about the main cancer symptoms
                    elif intent.get('direct_access'):
                        responses = intent['responses']
                        print("CHANCO: " + random.choice(responses))
                        
                    else:
                        print("CHANCO: " + random.choice(default_responses))
        
        #if not so confident - print a default text
        else :
            print("CHANCO: " + random.choice(default_responses))
        print()

In [154]:
chat("bro")

Hi bro, how can I help you today?
You: your name
your name
(132,)
(1, 132)
[1.33102707e-09 2.59217840e-24 2.14455226e-17 8.19575268e-20
 2.30266584e-09 4.90438912e-10 1.50575689e-12 1.00000000e+00
 3.24646365e-23 1.60915150e-17 1.18970914e-17 7.25760251e-12
 9.50051669e-12]
7
name
CHANCO: You can call me CHANCO (Chatbot + ONCO :P). Hope it sounds catch!

You: no
no
(132,)
(1, 132)
[4.5080282e-09 6.1716271e-08 2.6714752e-06 3.5676405e-07 2.4361632e-05
 1.5653474e-06 2.4859219e-08 5.6803173e-10 9.9988484e-01 6.4055384e-05
 5.7014784e-08 9.8560395e-09 2.2068485e-05]
8
name_no
CHANCO: For reals?? :'[

You: your name
your name
(132,)
(1, 132)
[1.33102707e-09 2.59217840e-24 2.14455226e-17 8.19575268e-20
 2.30266584e-09 4.90438912e-10 1.50575689e-12 1.00000000e+00
 3.24646365e-23 1.60915150e-17 1.18970914e-17 7.25760251e-12
 9.50051669e-12]
7
name
CHANCO: You can call me CHANCO (Chatbot + ONCO :P). Hope it sounds catch!

You: yes
yes
(132,)
(1, 132)
[9.5950103e-10 2.1272160e-09 2.7612256e-07 